In [1]:
import tensorflow as tf
import numpy as np
tf.reset_default_graph()

/Users/eabg97/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/eabg97/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Loading file
filename = "../sherlock.txt"
raw_text = open(filename).read()

In [3]:
# Create mapping of unique chars of integers
chars = sorted(set(raw_text))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [4]:
# Total sizes
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total characters", n_chars)
print("Total vocabulary", n_vocab)

Total characters 22157
Total vocabulary 75


In [5]:
# Preparing dataset
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i : i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total patterns", n_patterns)

Total patters 22057


In [6]:
# Reshape X to be [samples, time steps, features]
X = tf.reshape(dataX, (n_patterns, seq_length, 1))
X = X / n_vocab

# one hot encode the output variable
y = tf.keras.utils.to_categorical(dataY)
y = tf.convert_to_tensor(y, dtype=tf.int32)

In [7]:
# Parameters
dataset = n_chars
output_dim = n_vocab
EPOCHS = 500
BATCH_SIZE = 1000
learning_rate = 0.0001
patience = 10000

In [8]:
# Creating the variables of the dataset
dX = tf.placeholder(dtype=tf.float32, shape=[None, seq_length, 1])
dY = tf.placeholder(dtype=tf.float32, shape=[None, n_vocab])

In [9]:
# Implementing RNN
from tensorflow.contrib import rnn
def RNN(x, rnn_size=256, dropout=0.2, batch_size=64):
    with tf.variable_scope('RNN'):
        cell = rnn.BasicRNNCell(rnn_size, activation=tf.nn.relu)
        if dropout > 0.0:
            cell = rnn.DropoutWrapper(cell, output_keep_prob= 1. - dropout)
        initial_rnn_state = cell.zero_state(batch_size, dtype=tf.float32)
        outputs, final_rnn_state = tf.nn.dynamic_rnn(cell, x, initial_state=initial_rnn_state, dtype=tf.float32)
    outputs = tf.transpose(outputs, [1, 0, 2])
    last = outputs[-1]
    return last

In [10]:
# Implementing Softmax for Output
def softmax(input_, out_dim, scope=None):
    with tf.variable_scope(scope or 'softmax'):
        W = tf.get_variable('W', [input_.get_shape()[1], out_dim])
        b = tf.get_variable('b', [out_dim])
    return tf.nn.softmax(tf.matmul(input_, W) + b)

In [11]:
# Creating model
logits = RNN(dX, batch_size=BATCH_SIZE)
result = softmax(logits, output_dim)

In [12]:
# Loss
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=result, labels=dY))

# Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_op)

# Accuracy
predictions = tf.equal(tf.argmax(result, 1), tf.argmax(dY, 1))
acc = tf.reduce_mean(tf.cast(predictions, tf.float32))

In [13]:
# Executing RNN
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    datasetX, datasetY = sess.run([X, y])
    
    best_acc = 0.0
    DONE = False
    epoch = 0
    
    while epoch <= EPOCHS and not DONE:
        loss = 0.0
        batch = 1
        epoch += 1
        loops = dataset/BATCH_SIZE
        act_batch = 0
        
        while loops > 0:
            batch_x, batch_y = [], []
            max_batch = act_batch + BATCH_SIZE
            
            while min(dataset, max_batch) > act_batch:
                batch_x.append(datasetX[act_batch])
                batch_y.append(datasetY[act_batch])
                act_batch += 1
            
            batch_x = np.array(batch_x)
            batch_y = np.array(batch_y)
            
            loops -= 1
            _, loss_r, accuracy_r = sess.run([optimizer, loss_op, acc], feed_dict={dX: batch_x, dY: batch_y})
            
            loss += accuracy_r
            
            print(epoch, EPOCHS, loops, loss/loops, accuracy_r)

(1000, 100, 1)
(1000, 75)
1 500 21.157 0.0006144538577407431 0.013
(1000, 100, 1)
(1000, 75)
1 500 20.157 0.0009426006012985546 0.006
(1000, 100, 1)
(1000, 75)
1 500 19.157 0.0015660072172454101 0.011
(1000, 100, 1)
(1000, 75)
1 500 18.157 0.0020928567847524333 0.008
(1000, 100, 1)
(1000, 75)


KeyboardInterrupt: 